챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없다.

오픈 AI에서 제공하는 기본 API는 랭체인 없이 사용할 수 있지만 메모리를 지원하지 않자만, chatGPT에는 메모리가 탑재되어 있기 때문에 사용자가 이용할 떄 실제 사람과 대화를 예기하고 있다는 느낌을 들게 한다.


#### 1. Conversation Buffer Memory

단순히 이전 대화 내용 전체를 저장하는 메모리이지만 이 메모리의 단점은 대화 내용이 길어질수록 메모리도 계속 커지니까 비효울적이다.


#### 2. Conversation Buffer Window

시간에 따른 대화의 상호 작용 목록을 유지한다. 마지막 K개의 상호 작용만 사용한다. Conversation Buffer Memory의 단점을 보완해주지지만, 챗봇이 예전 대화를 기억하기 힘들며 최근에 일어난 대화에만 집중한다.

#### 3. Conversation Summary Memory

llm을 사용하는 메모리로 ChatOpenAI로부터 chat_model llm을 import한다. 이 메모리는 message를 그대로 저장하는 것이 아니라 conversation의 요약을 자체적으로 해준다. <br>
초반에는 짧은 메세지를 주고 받기 때문에 정보를 압축하지는 않지만 시간이 지나면서 많은 대화를 나눌수록 쌓인 정보가 많아졌기 때문에 대화가 진행되는 대로 요약하고 현재 요약 내용을 메모리에 저장한다.<br>


#### 4. Conversation Summary Buffer Memory

Conversation Summary Memory와 Conversation Buffer Memory를 결합한 메모리이다.
메모리에 보내온 메시지의 수만큼 메시지를 저장한다. 정해놓은 메시지의 수의 limit를 넘어서면 오래된 메시지들을 요약한다.



In [4]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    
def get_history():
    return memory.load_memory_variables({})

In [7]:
add_message("Hi I'm John, I live in South Korea", "Wow that is so cool!")

In [8]:
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [9]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [10]:
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [11]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [12]:
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [19]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [20]:
get_history()

{'history': [SystemMessage(content='John introduces himself as living in South Korea. The AI responds with enthusiasm, finding it cool.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}